In [1]:
# _*_ coding: utf-8 _*_
# @Time: 2022/02/11 10:05 
# @Author: yuyongsheng
# @Software: PyCharm
# @Description: 东方肝胆-肝门胆管癌：1. 数据处理

# 原始数据预处理

## 导入程序包

In [2]:
# 导入程序包
import pymysql as MySQLDB
import pandas as pd
import numpy as np
import datetime

import re
import sys
import os
project_path = os.getcwd()
print(project_path)

D:\PycharmProjects\HCCA_dongfang_hospital


## 导入预定义函数

In [3]:
# 过滤文字
import numpy as np
def filter_string(df,feature):
    # 过滤文字!!!!!!!!!!!!!!!!!!!!!!!!!!!
    df=df[df[feature].str.contains('\d')]
    return df

In [4]:
# 过滤特殊字符
def filter_spec_character(x):
    if pd.isnull(x):
        return np.nan
    else:
        x = re.sub('<|>|＞|＜|\+|\/|-|cm|kg','',str(x))
        return x

In [5]:
# 过滤异常大值
def filter_huge_value(df,feature,median_value=1):
    # 过滤异常大值!!!!!!!!!!!!!!!!!!!!!!!!!!
    if not median_value:
        median_value=df[feature].median()
    df[feature]=df[feature].apply(lambda x: np.nan if x<0 else x if float(x) <= 100 * median_value else 100*median_value)
    df=df[df[feature].notnull()]
    return df

## 读取raw_data

In [6]:
# 从df_doctor_order读入数据
df_raw_data =pd.read_excel(project_path +'/data/raw_data/HCCA -2022_1原始数据_规范化.xlsx')
if 'Unnamed: 0' in df_raw_data.columns:
    df_raw_data = df_raw_data.drop(['Unnamed: 0'], axis=1)

In [7]:
print(df_raw_data.shape)

(483, 222)


### 院方要删除的变量

In [8]:
# 院方要删除的变量
df_raw_data=df_raw_data.drop(['民族','婚姻状况','职业','体表面积M2','主诉','肝炎病史（1=有，写具体，0=无）','肿瘤病史（1=有，写具体，0=无）',
                             'CA72-4 u/ml <6.9','CA153u/ml <25','CA24-2 u/ml <25','CA-50 u/ml <=30','铁蛋白 ng/ml 30-400',
                              '异常凝血酶原 mAU/ml <=40','乙肝表面抗体（阳性=1，阴性=0）','术后病理TNM分期','术前PVE（1=有，0=无）',
                              '术前放疗','手术方式','术中探查','切缘','临床诊断','病理诊断','报告日期','免疫组化'],axis=1)

In [9]:
print(df_raw_data.shape)

(483, 198)


In [10]:
df_raw_data.head()

,id,取样方法,性别,年龄（岁）,身高（cm）,体重（kg）,体重指数,发病时有无黄疸（1=有，0=无）,有无消瘦（有=1，无=0）,emaciation_day,...,SUOX,TFF3,Villin,WT-1,围手术期并发症【Clavien-Dindo分级系统】（I级、II级、IIIa级、IIIb级、IVa级、IVb级、V级）,围手术期死亡【术后90天内】（0=无；1=死亡）,后续治疗及随访,目前是否生存（1=存活；2=死亡）,死亡时间,生存时间(天)
0,1,手术标本,男,59.0,NaN,NaN,NaN,1,1，20天减轻3kg,20.0,...,NaN,NaN,0.0,NaN,Ⅱ，输血,0,术后服用替吉奥胶囊化疗，后来出现全身淋巴结肿大，活检提示炎症，后来又出现腹水，2019年初经...,1,NaN,NaN
1,2,手术标本,男,64.0,NaN,NaN,NaN,1,1，1月减轻5公斤,30.0,...,NaN,NaN,0.0,NaN,Ⅰ,NaN,术后予以PD-1（O药）使用6次，2019年8月复查发现区域淋巴结复发，肝内复发可疑，予以行...,2，2020年3月,NaN,NaN
2,3,手术标本,男,52.0,NaN,NaN,NaN,1,1，1月减轻10公斤,30.0,...,NaN,NaN,NaN,NaN,Ⅰ,0,2019-05-27至2019-6-6行射波刀放疗6次（针对中肝叶术区及肝门淋巴结），后行肝移植术,2，2019年12月,NaN,NaN
3,4,手术标本,男,65.0,NaN,NaN,NaN,0,1，减轻约2公斤,NaN,...,NaN,NaN,NaN,NaN,Ⅲa，术后胆道感染，行PTCD穿刺引流,0,反复胆道感染，最终导致死亡,2，2019年2月,NaN,NaN
4,5,手术标本,女,59.0,NaN,NaN,NaN,1,0,NaN,...,NaN,NaN,0.0,NaN,V级，术后多次大量输血，术后行胸腔、腹腔穿刺引流，后治疗效果差，多器官功能衰竭，自动出院,1，自动出院,NaN,2，术后20余天,NaN,20天


### 合并基因突变数据

In [11]:
gene_mutation_index=df_raw_data.columns.to_list().index('KRAS-Exon-2')
gene_mutation_col=['KRAS-Exon-2','KRAS-Exon-3','KRAS-Exon-4','BRAF-Exon-15','NRAS-Exon-2','NRAS-Exon-3','NRAS-Exon-4','PIK3CA-exon-20',
          'SOX2','APOBEC3B','BRCA2','CCND1','CCND3','CDK12','CDKN2A','CDKN2B','DOT1L','EPHA7','ERBB2','ETV1','FAT1','FGF3','FGF4',
          'FGF14','FGF19','FLI1','KDM5A','LRP1B','MAP2K4','MAP3K1','MED12','MET','MLH1（MMR相关蛋白表达）','MSH2（MMR相关蛋白表达）',
          'MSH6（MMR相关蛋白表达）','MTOR','NF2','PMS2（MMR相关蛋白表达）','PRKD1','RARA','RET','SF3B1','SMAD4','STK11-exon4','STK24',
          'TGFBR1','TOP2A','TP53','VEGFA']

In [12]:
gene_mutation_index
df_raw_data.insert(gene_mutation_index,'基因突变',np.nan)

In [13]:
for i in range(df_raw_data.shape[0]):
    if df_raw_data.loc[i,gene_mutation_col].isnull().sum() < len(gene_mutation_col):
        df_raw_data.loc[i,'基因突变']=1
    else:
        df_raw_data.loc[i,'基因突变']=0

In [14]:
# 删除基因突变列
df_raw_data=df_raw_data.drop(gene_mutation_col,axis=1)

In [15]:
print(df_raw_data.shape)

(483, 150)


In [16]:
df_raw_data.to_excel(project_path+'/data/processed_data/df_1.3.2_合并基因突变数据.xlsx')

### 删除缺失超过90%变量

In [17]:
# 保护变量
protect_col=['emaciation_day','微卫星不稳','TMB','PD-1','PD-L1(DAKO,22C3)','PD-L1(罗氏，SP263)','死亡时间','生存时间(天)']
col=[i for i in df_raw_data.columns if i not in protect_col]
for i in col:
    if df_raw_data[i].isnull().sum()/df_raw_data.shape[0] > 0.9:
        print(i)
        del df_raw_data[i]

CHD4
CYP2A6
ERCC3
FAM135B
HLA-DRB1
KEAP1
MYC
PTCH1
PTEN
PTK2
RB1
SLC2A2
SMARCA4-exon10
SULT1A1
UGT1A7
α1-AT
AKR1
AMACR
Anti-CRP
AQP-1
Arginase
B-catenin
CA19-9
Calretinin
CD3
CD10
CD20
CD31
CD34
CD68
CK
CK18
D2-40
Desmin
Epcam
FGF2
gal-4
GLy-3
GPC3
GS
HBME-1
HBsAg
Hep-1(-)
HER1
HER2
HMB45
HNF4a
HSP70
Ki67
Lysozyme
MC
MRP3
MUC-2
NSE
OPN
P40
P53
P62
P63
PCNA
PGM-1
PS6
S-100
SDHB
SMA
SUOX
TFF3
Villin
WT-1


In [18]:
print(df_raw_data.shape)

(483, 81)


In [19]:
df_raw_data.to_excel(project_path+'/data/processed_data/df_1.3.3_预处理原始数据.xlsx')

### 取样方法

In [20]:
# 手术标本标记为1，未手术标记为0
df_raw_data.rename(columns={'取样方法':'sampling'},inplace=True)
df_raw_data['sampling']=df_raw_data['sampling'].apply(lambda x: np.nan if pd.isnull(x) else
                                                              0 if re.match('.*?[无未].*?',str(x)) else
                                                              1 if re.match('.*?手术.*?',str(x)) else x) 

In [21]:
df_raw_data['sampling'].value_counts()

1.0    446
0.0     29
Name: sampling, dtype: int64

## 计算生存时间

In [22]:
aaa=df_raw_data[df_raw_data['生存时间(天)'].notnull()]
bbb=df_raw_data[df_raw_data['生存时间(天)'].isnull()]

### aaa生存时间处理

In [23]:
aaa.shape

(83, 81)

In [24]:
aaa.head()

,id,sampling,性别,年龄（岁）,身高（cm）,体重（kg）,体重指数,发病时有无黄疸（1=有，0=无）,有无消瘦（有=1，无=0）,emaciation_day,...,MUC-1,MOC-31,PD-1,"PD-L1(DAKO,22C3)",围手术期并发症【Clavien-Dindo分级系统】（I级、II级、IIIa级、IIIb级、IVa级、IVb级、V级）,围手术期死亡【术后90天内】（0=无；1=死亡）,后续治疗及随访,目前是否生存（1=存活；2=死亡）,死亡时间,生存时间(天)
4,5,1.0,女,59.0,NaN,NaN,NaN,1,0,NaN,...,0.0,NaN,NaN,NaN,V级，术后多次大量输血，术后行胸腔、腹腔穿刺引流，后治疗效果差，多器官功能衰竭，自动出院,1，自动出院,NaN,2，术后20余天,NaN,20天
25,26,1.0,女,64.0,NaN,NaN,NaN,1,体重下降约5kg,NaN,...,NaN,NaN,NaN,NaN,V，感染性休克，出现多器官功能衰竭，自动出院,1，自动出院,NaN,2，术后20天,NaN,20天
52,53,1.0,女,68.0,NaN,NaN,NaN,1,1月来减轻5公斤,30.0,...,1.0,NaN,0.0,0,V，死于肝功能衰竭,1,NaN,2，术后20天左右,NaN,20天
53,54,1.0,男,74.0,NaN,NaN,NaN,1,体重下降10kg,NaN,...,1.0,NaN,0.0,0,V，术后8天胆瘘，肾衰予以透析引流，14天死亡,1,NaN,2，术后14天,NaN,14天
55,56,1.0,女,55.0,NaN,NaN,NaN,0,0,NaN,...,1.0,NaN,NaN,NaN,Ⅴ级，腹腔出血，DSA效果欠佳，自动出院,1，自动出院,NaN,2，术后20天左右,NaN,20天


In [25]:
# 月转换为天
aaa['生存时间(天)']=aaa['生存时间(天)'].apply(lambda x: int(x.replace('月',''))*30 if re.match('\d*月',x) else int(x.replace('天','')))

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [26]:
aaa

,id,sampling,性别,年龄（岁）,身高（cm）,体重（kg）,体重指数,发病时有无黄疸（1=有，0=无）,有无消瘦（有=1，无=0）,emaciation_day,...,MUC-1,MOC-31,PD-1,"PD-L1(DAKO,22C3)",围手术期并发症【Clavien-Dindo分级系统】（I级、II级、IIIa级、IIIb级、IVa级、IVb级、V级）,围手术期死亡【术后90天内】（0=无；1=死亡）,后续治疗及随访,目前是否生存（1=存活；2=死亡）,死亡时间,生存时间(天)
4,5,1.0,女,59.0,NaN,NaN,NaN,1,0,NaN,...,0.0,NaN,NaN,NaN,V级，术后多次大量输血，术后行胸腔、腹腔穿刺引流，后治疗效果差，多器官功能衰竭，自动出院,1，自动出院,NaN,2，术后20余天,NaN,20
25,26,1.0,女,64.0,NaN,NaN,NaN,1,体重下降约5kg,NaN,...,NaN,NaN,NaN,NaN,V，感染性休克，出现多器官功能衰竭，自动出院,1，自动出院,NaN,2，术后20天,NaN,20
52,53,1.0,女,68.0,NaN,NaN,NaN,1,1月来减轻5公斤,30.0,...,1.0,NaN,0.0,0,V，死于肝功能衰竭,1,NaN,2，术后20天左右,NaN,20
53,54,1.0,男,74.0,NaN,NaN,NaN,1,体重下降10kg,NaN,...,1.0,NaN,0.0,0,V，术后8天胆瘘，肾衰予以透析引流，14天死亡,1,NaN,2，术后14天,NaN,14
55,56,1.0,女,55.0,NaN,NaN,NaN,0,0,NaN,...,1.0,NaN,NaN,NaN,Ⅴ级，腹腔出血，DSA效果欠佳，自动出院,1，自动出院,NaN,2，术后20天左右,NaN,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434,435,1.0,女,46.0,168.0,67.5,23.92,1,NaN,NaN,...,NaN,NaN,NaN,NaN,ⅢB，术后腹腔出血，再手术止血,0,NaN,2，术后69个月,NaN,2070
435,436,1.0,男,62.0,162.0,72.0,27.43,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,NaN,2，术后37个月,NaN,1110
438,439,1.0,女,68.0,150.0,48.0,21.33,1,1，1月内减轻5公斤,30.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2，术后3年去世,NaN,1080
479,480,1.0,女,55.0,NaN,NaN,NaN,0,0,NaN,...,NaN,NaN,NaN,NaN,无,0,NaN,2，生存3月,NaN,90


In [27]:
aaa.to_excel(project_path+'/data/processed_data/aaa.xlsx')

### bbb生存事件处理

In [28]:
bbb.shape

(400, 81)

In [29]:
bbb.head()

,id,sampling,性别,年龄（岁）,身高（cm）,体重（kg）,体重指数,发病时有无黄疸（1=有，0=无）,有无消瘦（有=1，无=0）,emaciation_day,...,MUC-1,MOC-31,PD-1,"PD-L1(DAKO,22C3)",围手术期并发症【Clavien-Dindo分级系统】（I级、II级、IIIa级、IIIb级、IVa级、IVb级、V级）,围手术期死亡【术后90天内】（0=无；1=死亡）,后续治疗及随访,目前是否生存（1=存活；2=死亡）,死亡时间,生存时间(天)
0,1,1.0,男,59.0,NaN,NaN,NaN,1,1，20天减轻3kg,20.0,...,1.0,NaN,0.0,0.05,Ⅱ，输血,0,术后服用替吉奥胶囊化疗，后来出现全身淋巴结肿大，活检提示炎症，后来又出现腹水，2019年初经...,1,NaN,NaN
1,2,1.0,男,64.0,NaN,NaN,NaN,1,1，1月减轻5公斤,30.0,...,1.0,NaN,0.0,0.02,Ⅰ,NaN,术后予以PD-1（O药）使用6次，2019年8月复查发现区域淋巴结复发，肝内复发可疑，予以行...,2，2020年3月,NaN,NaN
2,3,1.0,男,52.0,NaN,NaN,NaN,1,1，1月减轻10公斤,30.0,...,NaN,NaN,NaN,NaN,Ⅰ,0,2019-05-27至2019-6-6行射波刀放疗6次（针对中肝叶术区及肝门淋巴结），后行肝移植术,2，2019年12月,NaN,NaN
3,4,1.0,男,65.0,NaN,NaN,NaN,0,1，减轻约2公斤,NaN,...,NaN,NaN,NaN,NaN,Ⅲa，术后胆道感染，行PTCD穿刺引流,0,反复胆道感染，最终导致死亡,2，2019年2月,NaN,NaN
5,6,1.0,女,68.0,NaN,NaN,NaN,1,0,NaN,...,0.0,NaN,NaN,NaN,Ⅰ,0,NaN,2，2019年2月,NaN,NaN


In [30]:
# 提取死亡时间，并转换
bbb=bbb.reset_index(drop=True)
for i in range(bbb.shape[0]):
#     print(i)
    label=bbb.loc[i,'目前是否生存（1=存活；2=死亡）']
    if re.match('2,|2，',str(label)):
        label_split=re.split(',|，|年|月|日|号|-|/|\.|（',str(label).replace(' ',''))
        # 提取死亡时间
        if label_split[-1]=='':
            death_time='/'.join(label_split[1:-1])
        else:
            death_time='/'.join(label_split[1:])
        death_time=pd.to_datetime(death_time)
        bbb.loc[i,'死亡时间']=death_time
        # 调整手术时间格式
        surgery_date=bbb.loc[i,'手术日期']
        if not isinstance(surgery_date,datetime.datetime):
            surgery_date=pd.to_datetime(surgery_date)
#         print(type(surgery_date))
        try:
            bbb.loc[i,'生存时间(天)']=(death_time-surgery_date).days
        except:
            bbb.loc[i,'生存时间(天)']=np.nan

In [31]:
bbb.head()

,id,sampling,性别,年龄（岁）,身高（cm）,体重（kg）,体重指数,发病时有无黄疸（1=有，0=无）,有无消瘦（有=1，无=0）,emaciation_day,...,MUC-1,MOC-31,PD-1,"PD-L1(DAKO,22C3)",围手术期并发症【Clavien-Dindo分级系统】（I级、II级、IIIa级、IIIb级、IVa级、IVb级、V级）,围手术期死亡【术后90天内】（0=无；1=死亡）,后续治疗及随访,目前是否生存（1=存活；2=死亡）,死亡时间,生存时间(天)
0,1,1.0,男,59.0,NaN,NaN,NaN,1,1，20天减轻3kg,20.0,...,1.0,NaN,0.0,0.05,Ⅱ，输血,0,术后服用替吉奥胶囊化疗，后来出现全身淋巴结肿大，活检提示炎症，后来又出现腹水，2019年初经...,1,NaN,NaN
1,2,1.0,男,64.0,NaN,NaN,NaN,1,1，1月减轻5公斤,30.0,...,1.0,NaN,0.0,0.02,Ⅰ,NaN,术后予以PD-1（O药）使用6次，2019年8月复查发现区域淋巴结复发，肝内复发可疑，予以行...,2，2020年3月,2020-03-01 00:00:00,536
2,3,1.0,男,52.0,NaN,NaN,NaN,1,1，1月减轻10公斤,30.0,...,NaN,NaN,NaN,NaN,Ⅰ,0,2019-05-27至2019-6-6行射波刀放疗6次（针对中肝叶术区及肝门淋巴结），后行肝移植术,2，2019年12月,2019-12-01 00:00:00,325
3,4,1.0,男,65.0,NaN,NaN,NaN,0,1，减轻约2公斤,NaN,...,NaN,NaN,NaN,NaN,Ⅲa，术后胆道感染，行PTCD穿刺引流,0,反复胆道感染，最终导致死亡,2，2019年2月,2019-02-01 00:00:00,44
4,6,1.0,女,68.0,NaN,NaN,NaN,1,0,NaN,...,0.0,NaN,NaN,NaN,Ⅰ,0,NaN,2，2019年2月,2019-02-01 00:00:00,37


In [32]:
bbb.to_excel(project_path+'/data/processed_data/bbb.xlsx')

### 合并aaa+bbb

In [33]:
df_survive=pd.concat([aaa,bbb],axis=0).reset_index(drop=True)

In [34]:
# 计算生存时间(月)
import math
df_survive['生存时间(月)']=df_survive['生存时间(天)'].apply(lambda x: math.ceil(x/30) if not pd.isnull(x) else x)

In [35]:
df_survive=df_survive.drop(['死亡时间'],axis=1)

In [36]:
df_survive.shape

(483, 81)

In [37]:
df_survive.head()

,id,sampling,性别,年龄（岁）,身高（cm）,体重（kg）,体重指数,发病时有无黄疸（1=有，0=无）,有无消瘦（有=1，无=0）,emaciation_day,...,MUC-1,MOC-31,PD-1,"PD-L1(DAKO,22C3)",围手术期并发症【Clavien-Dindo分级系统】（I级、II级、IIIa级、IIIb级、IVa级、IVb级、V级）,围手术期死亡【术后90天内】（0=无；1=死亡）,后续治疗及随访,目前是否生存（1=存活；2=死亡）,生存时间(天),生存时间(月)
0,5,1.0,女,59.0,NaN,NaN,NaN,1,0,NaN,...,0.0,NaN,NaN,NaN,V级，术后多次大量输血，术后行胸腔、腹腔穿刺引流，后治疗效果差，多器官功能衰竭，自动出院,1，自动出院,NaN,2，术后20余天,20,1.0
1,26,1.0,女,64.0,NaN,NaN,NaN,1,体重下降约5kg,NaN,...,NaN,NaN,NaN,NaN,V，感染性休克，出现多器官功能衰竭，自动出院,1，自动出院,NaN,2，术后20天,20,1.0
2,53,1.0,女,68.0,NaN,NaN,NaN,1,1月来减轻5公斤,30.0,...,1.0,NaN,0.0,0,V，死于肝功能衰竭,1,NaN,2，术后20天左右,20,1.0
3,54,1.0,男,74.0,NaN,NaN,NaN,1,体重下降10kg,NaN,...,1.0,NaN,0.0,0,V，术后8天胆瘘，肾衰予以透析引流，14天死亡,1,NaN,2，术后14天,14,1.0
4,56,1.0,女,55.0,NaN,NaN,NaN,0,0,NaN,...,1.0,NaN,NaN,NaN,Ⅴ级，腹腔出血，DSA效果欠佳，自动出院,1，自动出院,NaN,2，术后20天左右,20,1.0


In [38]:
df_survive.to_excel(project_path+'/data/processed_data/df_1.4.3_提取生存时间.xlsx')

In [39]:
df_survive['生存时间(天)']=df_survive['生存时间(天)'].apply(lambda x: int(x) if not pd.isnull(x) else x)
df_survive['生存时间(天)'].describe()

count     368.000000
mean      659.720109
std       637.379958
min         0.000000
25%       180.000000
50%       425.500000
75%       942.000000
max      2910.000000
Name: 生存时间(天), dtype: float64

In [40]:
df_survive['生存时间(月)'].describe()

count    368.000000
mean      22.380435
std       21.275201
min        0.000000
25%        6.000000
50%       15.000000
75%       32.000000
max       97.000000
Name: 生存时间(月), dtype: float64

## 人口学特征清洗

### gender

In [41]:
df_survive.rename(columns={'性别':'gender',
                           '年龄（岁）':'age'},inplace=True)
# 数据转换：男1，女0
df_survive['gender']=df_survive['gender'].apply(lambda x: np.nan if pd.isnull(x) else
                                                          0 if str(x)=='2' else
                                                          1 if str(x)=='1' else
                                                          1 if re.match('男',str(x)) else
                                                          0 if re.match('女',str(x)) else x)

In [42]:
df_survive.head()

,id,sampling,gender,age,身高（cm）,体重（kg）,体重指数,发病时有无黄疸（1=有，0=无）,有无消瘦（有=1，无=0）,emaciation_day,...,MUC-1,MOC-31,PD-1,"PD-L1(DAKO,22C3)",围手术期并发症【Clavien-Dindo分级系统】（I级、II级、IIIa级、IIIb级、IVa级、IVb级、V级）,围手术期死亡【术后90天内】（0=无；1=死亡）,后续治疗及随访,目前是否生存（1=存活；2=死亡）,生存时间(天),生存时间(月)
0,5,1.0,0.0,59.0,NaN,NaN,NaN,1,0,NaN,...,0.0,NaN,NaN,NaN,V级，术后多次大量输血，术后行胸腔、腹腔穿刺引流，后治疗效果差，多器官功能衰竭，自动出院,1，自动出院,NaN,2，术后20余天,20.0,1.0
1,26,1.0,0.0,64.0,NaN,NaN,NaN,1,体重下降约5kg,NaN,...,NaN,NaN,NaN,NaN,V，感染性休克，出现多器官功能衰竭，自动出院,1，自动出院,NaN,2，术后20天,20.0,1.0
2,53,1.0,0.0,68.0,NaN,NaN,NaN,1,1月来减轻5公斤,30.0,...,1.0,NaN,0.0,0,V，死于肝功能衰竭,1,NaN,2，术后20天左右,20.0,1.0
3,54,1.0,1.0,74.0,NaN,NaN,NaN,1,体重下降10kg,NaN,...,1.0,NaN,0.0,0,V，术后8天胆瘘，肾衰予以透析引流，14天死亡,1,NaN,2，术后14天,14.0,1.0
4,56,1.0,0.0,55.0,NaN,NaN,NaN,0,0,NaN,...,1.0,NaN,NaN,NaN,Ⅴ级，腹腔出血，DSA效果欠佳，自动出院,1，自动出院,NaN,2，术后20天左右,20.0,1.0


### 计算BMI

In [43]:
df_survive.rename(columns={'身高（cm）':'height',
                          '体重（kg）':'weight',
                          '体重指数':'BMI'},inplace=True)

In [44]:
df_survive['BMI']=[round(x*10000/y/y,2) 
              for (x,y) in zip(df_survive['weight'].astype('float').tolist(),df_survive['height'].astype('float').tolist())]

In [45]:
print(df_survive.shape)

(483, 81)


In [46]:
df_survive.head()

,id,sampling,gender,age,height,weight,BMI,发病时有无黄疸（1=有，0=无）,有无消瘦（有=1，无=0）,emaciation_day,...,MUC-1,MOC-31,PD-1,"PD-L1(DAKO,22C3)",围手术期并发症【Clavien-Dindo分级系统】（I级、II级、IIIa级、IIIb级、IVa级、IVb级、V级）,围手术期死亡【术后90天内】（0=无；1=死亡）,后续治疗及随访,目前是否生存（1=存活；2=死亡）,生存时间(天),生存时间(月)
0,5,1.0,0.0,59.0,NaN,NaN,NaN,1,0,NaN,...,0.0,NaN,NaN,NaN,V级，术后多次大量输血，术后行胸腔、腹腔穿刺引流，后治疗效果差，多器官功能衰竭，自动出院,1，自动出院,NaN,2，术后20余天,20.0,1.0
1,26,1.0,0.0,64.0,NaN,NaN,NaN,1,体重下降约5kg,NaN,...,NaN,NaN,NaN,NaN,V，感染性休克，出现多器官功能衰竭，自动出院,1，自动出院,NaN,2，术后20天,20.0,1.0
2,53,1.0,0.0,68.0,NaN,NaN,NaN,1,1月来减轻5公斤,30.0,...,1.0,NaN,0.0,0,V，死于肝功能衰竭,1,NaN,2，术后20天左右,20.0,1.0
3,54,1.0,1.0,74.0,NaN,NaN,NaN,1,体重下降10kg,NaN,...,1.0,NaN,0.0,0,V，术后8天胆瘘，肾衰予以透析引流，14天死亡,1,NaN,2，术后14天,14.0,1.0
4,56,1.0,0.0,55.0,NaN,NaN,NaN,0,0,NaN,...,1.0,NaN,NaN,NaN,Ⅴ级，腹腔出血，DSA效果欠佳，自动出院,1，自动出院,NaN,2，术后20天左右,20.0,1.0


In [47]:
df_survive.to_excel(project_path+'/data/processed_data/df_1.5.2_计算BMI.xlsx')

In [48]:
df_survive.shape

(483, 81)

## 病史清洗

In [49]:
df_history=df_survive.copy()

### 黄疸jaundice

In [50]:
df_history.rename(columns={'发病时有无黄疸（1=有，0=无）':'jaundice'},inplace=True)

In [51]:
df_history['jaundice']=df_history['jaundice'].apply(lambda x: 1 if re.match('有|轻度',str(x)) else 
                                                               0 if re.match('未知',str(x)) else x)

In [52]:
df_history['jaundice'].value_counts()

1    401
0     59
1     17
0      6
Name: jaundice, dtype: int64

### 有无消瘦

In [53]:
emaciation_index=df_history.columns.to_list().index('emaciation_kg')
df_history.insert(emaciation_index+1,'emaciation','')
df_history.insert(emaciation_index+1,'emaciation/month','')

In [54]:
# 1月下降体重
df_history['emaciation/month']=[round(y*30/x,2) 
              for (x,y) in zip(df_history['emaciation_day'].astype('float').tolist(),df_history['emaciation_kg'].astype('float').tolist())]

In [55]:
df_history['emaciation/month'].value_counts()

5.00     17
10.00    15
2.50      8
3.00      8
6.00      5
7.50      3
4.50      3
1.00      3
2.00      3
8.57      2
3.75      2
8.00      2
7.00      2
9.00      1
4.00      1
7.14      1
3.33      1
1.50      1
0.33      1
0.58      1
1.67      1
1.33      1
6.67      1
0.67      1
3.57      1
2.67      1
6.43      1
10.50     1
Name: emaciation/month, dtype: int64

In [56]:
# 标签：<=2kg,0; 2kg<x<5kg,1; >=5kg,2
df_history['emaciation']=df_history['emaciation/month'].apply(lambda x: 0 if x<=2 else
                                                                        1 if 2<x<5 else
                                                                        2 if x>=5 else x)

In [57]:
# 补充标签0
for i in range(df_history.shape[0]):
    if str(df_history.loc[i,'有无消瘦（有=1，无=0）']) == '0':
#         print(i)
        df_history.loc[i,'emaciation']=0

In [58]:
# 删除消瘦计算列
df_history=df_history.drop(['有无消瘦（有=1，无=0）','emaciation_day','emaciation_kg','emaciation/month'],axis=1)

In [59]:
df_history.to_excel(project_path+'/data/processed_data/df_1.6.2_有无消瘦数据清洗.xlsx')

### 呼吸系统疾病

In [60]:
df_history.rename(columns={'既往呼吸系统病史（1=有，写具体，0=无）':'breath_disease'},inplace=True)
df_history['breath_disease']=df_history['breath_disease'].apply(lambda x: np.nan if pd.isnull(x) else
                                                                           0 if str(x)=='0' else 1)

In [61]:
df_history['breath_disease'].value_counts()

0.0    470
1.0     12
Name: breath_disease, dtype: int64

In [62]:
df_history.to_excel(project_path+'/data/processed_data/df_1.6.3_呼吸系统疾病.xlsx')

### 心血管疾病

In [63]:
df_history.rename(columns={'既往心脑血管系统病史（1=有，写具体，0=无）':'cardio_disease'},inplace=True)
df_history['cardio_disease']=df_history['cardio_disease'].apply(lambda x: np.nan if pd.isnull(x) else
                                                                           0 if str(x)=='0' else 1)

In [64]:
df_history['cardio_disease'].value_counts()

0.0    372
1.0    110
Name: cardio_disease, dtype: int64

In [65]:
df_history.to_excel(project_path+'/data/processed_data/df_1.6.4_心脑血管疾病.xlsx')

### 非胆道消化系统疾病

In [66]:
df_history.rename(columns={'既往非胆道消化系统病史（1=有，写具体，0=无）':'nbdd'},inplace=True)
df_history['nbdd']=df_history['nbdd'].apply(lambda x: np.nan if pd.isnull(x) else
                                                       0 if str(x)=='0' else 1)

In [67]:
df_history['nbdd'].value_counts()

0.0    433
1.0     49
Name: nbdd, dtype: int64

In [68]:
df_history.to_excel(project_path+'/data/processed_data/df_1.6.5_非胆道消化系统疾病.xlsx')

### 泌尿系统疾病

In [69]:
df_history.rename(columns={'既往泌尿系统病史（1=有，写具体，0=无）':'urinary_disease'},inplace=True)
df_history['urinary_disease']=df_history['urinary_disease'].apply(lambda x: np.nan if pd.isnull(x) else
                                                                           0 if str(x)=='0' else 1)

In [70]:
df_history['urinary_disease'].value_counts()

0.0    469
1.0     11
Name: urinary_disease, dtype: int64

In [71]:
df_history.to_excel(project_path+'/data/processed_data/df_1.6.6_泌尿系统疾病.xlsx')

### 血液系统疾病

In [72]:
df_history=df_history.drop(['既往血液系统病史（1=有，写具体，0=无）'],axis=1)

In [73]:
df_history.shape

(483, 78)

### 内分泌系统疾病

In [74]:
df_history.rename(columns={'既往内分泌系统及代谢系统病史（1=有，写具体，0=无）':'endocrine_disease'},inplace=True)
df_history['endocrine_disease']=df_history['endocrine_disease'].apply(lambda x: np.nan if pd.isnull(x) else
                                                                           0 if str(x)=='0' else 1)

In [75]:
df_history['endocrine_disease'].value_counts()

0.0    420
1.0     61
Name: endocrine_disease, dtype: int64

In [76]:
df_history.to_excel(project_path+'/data/processed_data/df_1.6.8_内分泌系统疾病.xlsx')

### 风湿免疫系统疾病

In [77]:
df_history=df_history.drop(['既往风湿免疫系统病史（1=有，写具体，0=无）'],axis=1)

In [78]:
df_history.shape

(483, 77)

### 胆道系统疾病

In [79]:
df_history.rename(columns={'既往胆道系统病史或本次发现胆道结石（1=有，写具体，0=无）':'biliary_disease'},inplace=True)
df_history['biliary_disease']=df_history['biliary_disease'].apply(lambda x: np.nan if pd.isnull(x) else
                                                                           0 if str(x)=='0' else 1)

In [80]:
df_history['biliary_disease'].value_counts()

0.0    406
1.0     75
Name: biliary_disease, dtype: int64

In [81]:
df_history.to_excel(project_path+'/data/processed_data/df_1.6.10_胆道系统疾病.xlsx')

### 其他疾病病史

In [82]:
df_history.rename(columns={'既往其他病史（1=有，写具体，0=无）':'other_disease'},inplace=True)
df_history['other_disease']=df_history['other_disease'].apply(lambda x: np.nan if pd.isnull(x) else
                                                                           0 if str(x)=='0' else 1)

In [83]:
df_history['other_disease'].value_counts()

0.0    369
1.0    110
Name: other_disease, dtype: int64

In [84]:
df_history.to_excel(project_path+'/data/processed_data/df_1.6.11_其他疾病病史.xlsx')

### 吸烟史

In [85]:
# 少量吸烟标记为0
df_history.rename(columns={'吸烟史（1=有，0=无）':'smoke'},inplace=True)
df_history['smoke']=df_history['smoke'].apply(lambda x: np.nan if pd.isnull(x) else
                                                        0 if str(x)=='0' else
                                                        0 if re.match('.*?[少无否认].*',str(x)) else 1)

In [86]:
df_history['smoke'].value_counts()

0    348
1    135
Name: smoke, dtype: int64

In [87]:
df_history.to_excel(project_path+'/data/processed_data/df_1.6.12_吸烟史.xlsx')

### 饮酒史

In [88]:
# 少量饮酒、偶尔饮酒标记为0
df_history.rename(columns={'饮酒史（1=有，0=无）':'drinking'},inplace=True)
df_history['drinking']=df_history['drinking'].apply(lambda x: np.nan if pd.isnull(x) else
                                                        0 if str(x)=='0' else
                                                        0 if re.match('.*?[少偶无否认].*',str(x)) else 1)

In [89]:
df_history['drinking'].value_counts()

0.0    368
1.0     91
Name: drinking, dtype: int64

In [90]:
df_history.to_excel(project_path+'/data/processed_data/df_1.6.13_饮酒史.xlsx')

### 家族史

In [91]:
df_history.rename(columns={'家族史':'family_history'},inplace=True)
df_history['family_history']=df_history['family_history'].apply(lambda x: np.nan if pd.isnull(x) else
                                                        0 if str(x)=='0' else
                                                        0 if re.match('无',str(x)) else 1)

In [92]:
df_history['family_history'].value_counts()

0.0    447
1.0     29
Name: family_history, dtype: int64

In [93]:
df_history.to_excel(project_path+'/data/processed_data/df_1.6.14_家族史.xlsx')

### 血型

In [94]:
# A型血标记1，B型血标记为2，AB型血标记为3，O型血标记为0
df_history.rename(columns={'血型':'blood_type'},inplace=True)
df_history['blood_type']=df_history['blood_type'].apply(lambda x: np.nan if pd.isnull(x) else
                                                        3 if re.match('AB.*?',str(x)) else
                                                        2 if re.match('B.*?',str(x)) else
                                                        1 if re.match('A.*?',str(x)) else
                                                        0 if re.match('[Oo0].*?',str(x)) else np.nan)

In [95]:
df_history['blood_type'].value_counts()

1.0    44
0.0    40
2.0    35
3.0    18
Name: blood_type, dtype: int64

In [96]:
df_history.to_excel(project_path+'/data/processed_data/df_1.6.15_血型.xlsx')

## 术前检测

In [97]:
df_test=df_history.copy()

In [98]:
# 删除总胆汁酸
df_test=df_test.drop(['总胆汁酸（umol/L）<=12'],axis=1)
# 14个变量重命名
df_test=df_test.rename(columns={'白细胞计数（*109/L）（3.5-9.5）':'WBC',
                                '血红蛋白（g/L)（115-150）':'HGB',
                                '血小板计数（*109/L）（125-350）':'PLT',
                                '总胆红素（umol/L）（<=23）':'TB',
                                '直接胆红素（umol/L）(<=8)':'DB',
                                '总蛋白（g/L）65-85':'TP',
                                '白蛋白（g/L）40-55':'ALB',
                                '球蛋白（g/L）20-40':'LG',
                                '白球比 1.2-2.4':'AG',
                                '前白蛋白（mg/L）170-420':'PAB',
                                '谷丙转氨酶（U/L）7-40':'ALT',
                                '谷草转氨酶（U/L）13-35':'AST',
                                'γ-GT（U/L）7-45':'GT',
                                '碱性磷酸酶（U/L）35-135':'ALP'})

In [99]:
df_test['TB']=df_test['TB'].apply(lambda x: str(x).replace('（PTCD后）','').replace('已行PTCD',''))

In [100]:
df_test.shape

(483, 76)

In [101]:
df_test.to_excel(project_path+'/data/processed_data/df_1.7_术前检测.xlsx')

## 基因水平

### 肿瘤指标

In [102]:
df_tumor=df_test.copy()

In [103]:
### CA19-9，CEA等
df_tumor=df_tumor.rename(columns={'AFP ng/ml<20':'tumor_AFP',
                                'CEAng/ml <10':'tumor_CEA',
                                'CA19-9 u/ml <39':'tumor_CA199',
                                'CA125 u/ml <35':'tumor_CA125'})

#### AFP异常值处理

In [104]:
# 删除文字
df_tumor['tumor_AFP']=df_tumor['tumor_AFP'].apply(lambda x: np.nan if pd.isnull(x) else
                                                    x if re.match('.*?\d+.*?',str(x)) else np.nan)
# 删除特殊符号
df_tumor['tumor_AFP']=df_tumor['tumor_AFP'].apply(filter_spec_character)

In [105]:
df_tumor['tumor_AFP'].unique()

array(['2.8', '3.9', '5.2', '5.3', '4.3', '4.8', '4.1', '3', '7.2', '1.6',
       '2.1', nan, '11', '3.2', '7.1', '6', '2.5', '2.4', '4.5', '1.3',
       '3.1', '7.3', '2.7', '3.7', '2.6', '11.4', '3.4', '2.2', '4.9',
       '4.7', '1', '2.9', '1.5', '1.4', '0.1', '2', '4.6', '4.4', '1.8',
       '5.4', '4', '1.2', '1.7', '3.5', '3.6', '6.6', '2.3', '9.18',
       '5.7', '5.5', '3.8', '1.9', '4.2', '7.4', '5.9', '14.7', '18.3',
       '6.9', '8.3', '10.2', '3.3', '7', '12.2', '407.1', '6.8', '5',
       '6.7', '5.1', '6.2', '14.8', '0.9', '11.8', '0.7', '8.8', '3.44',
       '3.07', '8.2', '3.06', '19.5', '14.3', '6.3', '6.5', '1.1', '7.7',
       '3.11', '1.97', '12.3', '5.84', '4.62', '3.12', '3.79', '7.56',
       '3.83', '1.47', '2.62', '3.52', '5.68', '12.1', '9', '4.15',
       '3.53', '6.63', '1.24', '2.93', '6.84', '6.91', '4.53', '1.76',
       '1.02', '2.29', '3.65', '1210'], dtype=object)

In [106]:
df_tumor[df_tumor['tumor_AFP'].notnull()].shape

(464, 76)

In [107]:
df_tumor.to_excel(project_path+'/data/processed_data/df_1.8.1.1_AFP处理.xlsx')

#### CEA异常值处理

In [108]:
# 删除文字
df_tumor['tumor_CEA']=df_tumor['tumor_CEA'].apply(lambda x: np.nan if pd.isnull(x) else
                                                x if re.match('.*?\d+.*?',str(x)) else np.nan)

In [109]:
df_tumor['tumor_CEA'].unique()

array([0.6, 2.3, 2.7, 1.9, 12.5, 3.3, '3.9', '4.4', '3.1', '4.7', '1.8',
       '97.7', '7.3', '7.5', 2.1, 3.4, 6.8, 1.4, 3.5, 24.2, 4.6, 4.3, 1.3,
       4.4, 5.1, 3.7, 2.9, 3.9, 4.1, 12.7, 2.5, 22.2, 4.2, 67.2, 7.8, 3.6,
       7.6, 3, 5.6, 5.3, 2.2, 8.3, 9.1, 103.4, 4.7, 39.2, 1.6, 4, 5.7,
       21.6, 15.4, 1.7, nan, 9.9, 10.9, 2.6, 2.8, 10.39, 6.2, 5.8, 2.4,
       3.1, 8.8, 3.2, 5, 1.1, 105.4, 0.7, 0.8, 14.1, 4.5, 1.5, 1.2, 12,
       8.2, 4.8, 7.3, 26, 1, 8.7, 122.6, 2, 8.4, 4.9, 6.9, '3.0', '6.2',
       '2.3', '0.7', '2.0', '3.2', '4.5', '5.5', '18.1', '2.1', '1.3',
       '4.8', '4.9', '14.7', '2.2', '4.6', '2.6', '2.8', '0.3', '1.9',
       '12.8', '1.5', '21.2', '1.2', '5.3', '10.5', '55.4', '6.7', '2',
       '9.7', '3.7', '3.3', '3.8', '20.6', '5.6', '2.9', '1.4', '3.4',
       '8.8', '6.1', '9', '3', '6.4', '2.5', '4', '2.7', '5.8', '4.1',
       '9.9', '34.7', '8.2', '10.2', '1', '0.9', '7.7', '10', '0.6', '7',
       27.2, 1.94, 3.04, 4.38, 5.9, 276, 1.8, 5.2, 10.8, 3.

In [110]:
df_tumor[df_tumor['tumor_CEA'].notnull()].shape

(471, 76)

#### CA19-9异常处理

In [111]:
# 删除特殊符号
df_tumor['tumor_CA199']=df_tumor['tumor_CA199'].apply(filter_spec_character)
# 最大值设定为1000
df_tumor['tumor_CA199']=df_tumor['tumor_CA199'].apply(lambda x: 1000 if float(x)>1000 else x)

In [112]:
df_tumor['tumor_CA199'].unique()

array(['287.8', '1000', '614.5', '215.3', '3.7', '71.9', '341', '134.9',
       '113.3', '64.9', '289', '215.9', '99.4', '128.7', '1', '201.4',
       '297.5', '359', '58.2', '23.8', '45.6', '106.6', '133.7', '292.6',
       '949', '493.1', '78.1', '2.9', '604.1', '330.9', '1.8', '165.9',
       '29.7', '48.1', '293', '61.5', '8.8', '486.6', '0.7', '227.7',
       '121.5', '308.5', '944.7', '0.5', '558.2', '178.9', '775.3',
       '590.8', '658', '155', '176.5', '177.9', '381.8', '856.9', '29.9',
       '162.5', '39.6', '33.1', '292.7', '39.3', '309', '38.7', '38.4',
       '86.4', '880.6', '262.9', '918.6', '217.7', '1.7', '69.7', '10.2',
       '204.2', '206.2', '431.4', '63.6', '28.7', '12.1', '2.7', '86.2',
       '146.4', '10.1', '328.7', '407.3', '96.4', '73.9', '846.8', '75.3',
       '95.9', '124.6', '50', '143', '324', '659.2', '0.6', '334', '336',
       '150', '41.7', '199.7', '152.9', '209.5', '416.7', '106.5',
       '100.4', '56.4', '170.3', '2.1', '418', '18.2', '77.3', 

In [113]:
df_tumor[df_tumor['tumor_CA199'].notnull()].shape

(477, 76)

#### CA125异常值处理

In [114]:
df_tumor[df_tumor['tumor_CA125'].notnull()].shape

(319, 76)

#### 肿瘤最大径

In [115]:
# 肿瘤最大径（cm）
df_tumor.rename(columns={'肿瘤最大径（cm）':'tumor_size'},inplace=True)
# 3*3cm提取直径
df_tumor['tumor_size']=df_tumor['tumor_size'].apply(lambda x: re.findall(r'[*](\d+?)cm',str(x))[0] if '*' in str(x) else x)
# 剔除cm和?
df_tumor['tumor_size']=df_tumor['tumor_size'].apply(lambda x: str(x).replace('cm','').replace('?',''))

In [116]:
# 剔除cm
df_tumor['tumor_size']=df_tumor['tumor_size'].apply(filter_spec_character)

In [117]:
# 删除文字
df_tumor['tumor_size']=df_tumor['tumor_size'].apply(lambda x: np.nan if pd.isnull(x) else
                                        x if re.match('.*?\d+.*?',str(x)) else np.nan)

In [118]:
df_tumor['tumor_size'].unique()

array(['1', '3', '3.1', '1.8', '2.1', '2', '2.5', '4', nan, '6', '5',
       '1.2', '0', '8', '1.1', '1.5', '3.5', '7.2', '4.1', '4.2', '3.2',
       '3.4', '2.2', '5.8', '4.5', '2.8', '0.5', '1.3', '8.3'],
      dtype=object)

In [119]:
df_tumor[df_tumor['tumor_size'].notnull()].shape

(370, 76)

In [120]:
df_tumor.to_excel(project_path+'/data/processed_data/df_1.8.1_肿瘤指标.xlsx')

### 乙肝丙肝

In [121]:
df_HB=df_tumor.copy()

In [122]:
# 乙肝表面抗原（阳性=1，阴性=0）
df_HB.rename(columns={'乙肝表面抗原（阳性=1，阴性=0）':'HBsAg'},inplace=True)
df_HB['HBsAg']=df_HB['HBsAg'].apply(lambda x: np.nan if pd.isnull(x) else
                                                    0 if str(x)=='0' else 
                                                    1 if str(x)=='1' else x)

In [123]:
df_HB['HBsAg'].value_counts()

0.0    417
1.0     21
Name: HBsAg, dtype: int64

In [124]:
# 乙肝e抗原（阳性=1，阴性=0）
df_HB.rename(columns={'乙肝e抗原（阳性=1，阴性=0）':'HBeAg'},inplace=True)
df_HB['HBeAg']=df_HB['HBeAg'].apply(lambda x: np.nan if pd.isnull(x) else
                                                    0 if str(x)=='0' else 
                                                    1 if str(x)=='1' else x)

In [125]:
df_HB['HBeAg'].value_counts()

0.0    406
1.0      4
Name: HBeAg, dtype: int64

In [126]:
# 乙肝e抗体（阳性=1，阴性=0）
df_HB.rename(columns={'乙肝e抗体（阳性=1，阴性=0）':'HBeAb'},inplace=True)
df_HB['HBeAb']=df_HB['HBeAb'].apply(lambda x: np.nan if pd.isnull(x) else
                                                    0 if str(x)=='0' else 
                                                    1 if str(x)=='1' else x)

In [127]:
df_HB['HBeAb'].value_counts()

0.0    300
1.0    110
Name: HBeAb, dtype: int64

In [128]:
# 乙肝核心抗体（阳性=1，阴性=0）
df_HB.rename(columns={'乙肝核心抗体（阳性=1，阴性=0）':'HBcAb'},inplace=True)
df_HB['HBcAb']=df_HB['HBcAb'].apply(lambda x: np.nan if pd.isnull(x) else
                                                    0 if str(x)=='0' else 
                                                    1 if str(x)=='1' else x)

In [129]:
df_HB['HBcAb'].value_counts()

1.0    268
0.0    141
Name: HBcAb, dtype: int64

In [130]:
# 丙肝抗体（阳性=1，阴性=0）
df_HB.rename(columns={'丙肝抗体（阳性=1，阴性=0）':'HCVAb'},inplace=True)
df_HB['HCVAb']=df_HB['HCVAb'].apply(lambda x: np.nan if pd.isnull(x) else
                                                    0 if str(x)=='0' else 
                                                    1 if str(x)=='1' else x)

In [131]:
df_HB['HCVAb'].value_counts()

0.0    401
1.0      3
Name: HCVAb, dtype: int64

In [132]:
# 有无肝硬化（有=1，无=0）
df_HB.rename(columns={'有无肝硬化（有=1，无=0）':'LC'},inplace=True)
df_HB['LC']=df_HB['LC'].apply(lambda x: np.nan if pd.isnull(x) else
                                                    0 if str(x)=='0' else 
                                                    1 if str(x)=='1' else x)

In [133]:
df_HB['LC'].value_counts()

0.0    348
1.0     10
Name: LC, dtype: int64

In [134]:
df_HB.to_excel(project_path+'/data/processed_data/df_1.8.2_乙肝丙肝.xlsx')

### 术前术后分期

In [135]:
df_staging=df_HB.copy()

#### 术后AJCC-8分期

In [136]:
if re.match('.*?ⅢA|Ⅲa.*?','ⅢA'):
    print('true')

true


In [137]:
'''
术后病理AJCC-8分期
标签：0、I、II、III、IVA -- 0、1、2、3、4
'''
df_staging.rename(columns={'术后病理AJCC-8分期':'AJCC_8'},inplace=True)
df_staging['AJCC_8']=df_staging['AJCC_8'].apply(lambda x: np.nan if pd.isnull(x) else 
                                                        np.nan if str(x)=='未手术' else
                                                        0 if str(x)=='0' else 
                                                        1 if re.match('.*?Ⅰ.*?',str(x)) else
                                                        2 if re.match('.*?Ⅱ.*?',str(x)) else
                                                        3 if re.match('.*?Ⅲ.*?',str(x)) else
                                                        4 if re.match('.*?Ⅳ.*?',str(x)) else x)

In [138]:
df_staging['AJCC_8'].value_counts()

3.0    217
2.0    172
4.0     50
0.0      5
1.0      1
Name: AJCC_8, dtype: int64

#### 术前Gazzaniga T 分期

In [139]:
'''
术前Gazzaniga T 分期
标签：I、II、III、IV -- 1、2、3、4
'''
df_staging.rename(columns={'术前Gazzaniga T 分期':'Gazzaniga_T'},inplace=True)
df_staging['Gazzaniga_T']=df_staging['Gazzaniga_T'].apply(lambda x: np.nan if pd.isnull(x) else 
                                                        np.nan if str(x)=='未手术' else
                                                        1 if re.match('.*?Ⅰ|T1.*?',str(x)) else
                                                        2 if re.match('.*?Ⅱ.*?',str(x)) else
                                                        3 if re.match('.*?Ⅲ.*?',str(x)) else
                                                        4 if re.match('.*?Ⅳ.*?',str(x)) else x)

In [140]:
df_staging['Gazzaniga_T'].value_counts()

1.0    265
3.0     75
2.0     55
4.0     47
Name: Gazzaniga_T, dtype: int64

#### 术前MSKCC 分期

In [141]:
'''
术前MSKCC 分期
标签：I、II、III -- 1、2、3
'''
df_staging.rename(columns={'术前MSKCC 分期':'MSKCC'},inplace=True)
df_staging['MSKCC']=df_staging['MSKCC'].apply(lambda x: np.nan if pd.isnull(x) else 
                                                        np.nan if str(x)=='未手术' else
                                                        1 if re.match('.*?Ⅰ|T1.*?',str(x)) else
                                                        2 if re.match('.*?Ⅱ|T2.*?',str(x)) else
                                                        3 if re.match('.*?Ⅲ|T3.*?',str(x)) else x)

In [142]:
df_staging['MSKCC'].value_counts()

1.0    271
3.0    110
2.0     61
Name: MSKCC, dtype: int64

#### 术前Blumgart T分期

In [143]:
'''
术前Blumgart T分期
标签：I、II、III、IV -- 1、2、3、4
'''
df_staging.rename(columns={'术前Blumgart T分期':'Blumgart_T'},inplace=True)
df_staging['Blumgart_T']=df_staging['Blumgart_T'].apply(lambda x: np.nan if pd.isnull(x) else 
                                                        np.nan if str(x)=='未手术' else
                                                        1 if re.match('.*?Ⅰ|T1.*?',str(x)) else
                                                        2 if re.match('.*?Ⅱ|T2.*?',str(x)) else
                                                        3 if re.match('.*?Ⅲ|T3.*?',str(x)) else
                                                        4 if re.match('.*?Ⅳ|T4.*?',str(x)) else x)

In [144]:
df_staging['Blumgart_T'].value_counts()

1.0    270
4.0    110
3.0     41
2.0     21
Name: Blumgart_T, dtype: int64

#### 解剖学Bismuth-Corlette分型

In [145]:
'''
解剖学Bismuth-Corlette分型
标签：I、II、III、IV -- 1、2、3、4
'''
df_staging.rename(columns={'解剖学Bismuth-Corlette分型':'Bismuth_C'},inplace=True)
df_staging['Bismuth_C']=df_staging['Bismuth_C'].apply(lambda x: np.nan if pd.isnull(x) else 
                                                        1 if re.match('.*?Ⅰ.*?',str(x)) else
                                                        2 if re.match('.*?Ⅱ.*?',str(x)) else
                                                        3 if re.match('.*?Ⅲ|III.*?',str(x)) else
                                                        4 if re.match('.*?Ⅳ.*?',str(x)) else np.nan)

In [146]:
df_staging['Bismuth_C'].value_counts()

3.0    262
4.0     67
1.0     64
2.0     53
Name: Bismuth_C, dtype: int64

In [147]:
df_staging.to_excel(project_path+'/data/processed_data/df_1.8.3_术前术后分型.xlsx')

### 手术surgery

In [148]:
df_surgery=df_staging.copy()

#### 术前PTCD、ERCP等减黄（1=有，0=无）

In [149]:
if re.match('.*?E.*?PTCD.*?|.*?PTCD.*?E.*?','EM,203PTCD'):
    print('true')

true


In [150]:
'''
术前PTCD、ERCP等减黄（1=有，0=无）
标签:0、PTCD、ERCP、PTCD+ERCP --0、1、2、3
'''
df_surgery.rename(columns={'术前PTCD、ERCP等减黄（1=有，0=无）':'PTCD_ERCP'},inplace=True)
df_surgery['PTCD_ERCP']=df_surgery['PTCD_ERCP'].apply(lambda x: np.nan if pd.isnull(x) else
                                                                0 if str(x)=='0' else
                                                                1 if str(x)=='1' else
                                                                3 if re.match('.*?E.*?PTCD.*?|.*?PTCD.*?E.*?',str(x)) else
                                                                1 if re.match('.*?PTCD.*?',str(x)) else
                                                                2 if re.match('.*?E.*?',str(x)) else np.nan)

In [151]:
df_surgery['PTCD_ERCP'].value_counts()

1.0    252
0.0    155
2.0     31
3.0     15
Name: PTCD_ERCP, dtype: int64

#### 术中出血（ml）

In [152]:
'''
术中出血（ml）
出血少：定义为150；出血0定义为10ml
'''
df_surgery.rename(columns={'术中出血（ml）':'surgery_bleeding'},inplace=True)
df_surgery['surgery_bleeding']=df_surgery['surgery_bleeding'].apply(lambda x: np.nan if pd.isnull(x) else
                                                                10 if str(x)=='0' else
                                                                150 if re.match('[出血少不多]',str(x)) else
                                                                int(x) if re.match('\d+?',str(x)) else np.nan)

In [153]:
df_surgery[df_surgery['surgery_bleeding'].notnull()].shape

(431, 76)

In [154]:
df_surgery['surgery_bleeding'].value_counts()

150.0     74
200.0     73
300.0     63
400.0     51
500.0     40
600.0     28
100.0     28
800.0     20
700.0      8
1500.0     6
1000.0     6
50.0       6
1200.0     5
250.0      4
350.0      4
1600.0     2
80.0       2
2000.0     2
4000.0     2
1300.0     1
5.0        1
450.0      1
10.0       1
1400.0     1
20.0       1
750.0      1
Name: surgery_bleeding, dtype: int64

#### 输红细胞悬液（u）

In [155]:
# 输红细胞悬液（u）
df_surgery.rename(columns={'输红细胞悬液（u）':'surgery_CRCS'},inplace=True)
df_surgery['surgery_CRCS']=df_surgery['surgery_CRCS'].apply(lambda x: np.nan if pd.isnull(x) else
                                                            int(x) if re.match('\d+?',str(x)) else np.nan)

In [156]:
df_surgery[df_surgery['surgery_CRCS'].notnull()].shape

(430, 76)

In [157]:
df_surgery['surgery_CRCS'].value_counts()

0.0       310
4.0        42
2.0        28
3.0        20
6.0         6
300.0       5
8.0         5
5.0         5
600.0       3
12.0        2
1200.0      1
20.0        1
1.0         1
10.0        1
Name: surgery_CRCS, dtype: int64

#### 输血浆（ml）

In [158]:
# 输血浆（ml）
df_surgery.rename(columns={'输血浆（ml）':'surgery_plasm'},inplace=True)
df_surgery['surgery_plasm']=df_surgery['surgery_plasm'].apply(lambda x: np.nan if pd.isnull(x) else
                                                            int(x) if re.match('\d+?',str(x)) else np.nan)

In [159]:
df_surgery[df_surgery['surgery_plasm'].notnull()].shape

(428, 76)

In [160]:
df_surgery['surgery_plasm'].value_counts()

0.0       272
400.0      61
600.0      33
800.0      24
200.0      18
1000.0     10
300.0       3
1400.0      2
1200.0      2
1100.0      1
2800.0      1
1600.0      1
Name: surgery_plasm, dtype: int64

#### 冷沉淀（u）

In [161]:
# 冷沉淀（u）
df_surgery.rename(columns={'冷沉淀（u）':'surgery_CP'},inplace=True)

In [162]:
df_surgery['surgery_CP'].value_counts()

0.0     342
10.0      3
31.0      1
20.0      1
Name: surgery_CP, dtype: int64

#### 手术根治情况

In [163]:
# 手术根治情况（1=R0切除；2=R1切除；3=R2切除；4=剖腹探查、活检）
df_surgery.rename(columns={'手术根治情况（1=R0切除；2=R1切除；3=R2切除；4=剖腹探查、活检）':'surgery_result'},inplace=True)
df_surgery['surgery_result']=df_surgery['surgery_result'].apply(lambda x: np.nan if pd.isnull(x) else
                                                                1 if re.match('1',str(x)) else
                                                                2 if re.match('2',str(x)) else
                                                                3 if re.match('3',str(x)) else
                                                                4 if re.match('4',str(x)) else np.nan)

In [164]:
df_surgery['surgery_result'].value_counts()

1.0    305
4.0     65
3.0     31
2.0     29
Name: surgery_result, dtype: int64

In [165]:
df_surgery.to_excel(project_path+'/data/processed_data/df_1.8.4_手术surgery.xlsx')

### 基因突变

In [166]:
df_gene=df_surgery.copy()

#### 基因指标

In [167]:
df_gene.rename(columns={'基因突变':'gene_mutation'},inplace=True)

#### 微卫星不稳

In [168]:
'''
微卫星不稳
标签：MSS--0；MSI-H--1
'''
df_gene.rename(columns={'微卫星不稳':'gene_MSI'},inplace=True)
df_gene['gene_MSI']=df_gene['gene_MSI'].apply(lambda x: np.nan if pd.isnull(x) else
                                                        0 if re.match('MSS',str(x)) else
                                                        1 if re.match('MSI-H',str(x)) else np.nan)

In [169]:
df_gene['gene_MSI'].value_counts()

0.0    28
1.0     1
Name: gene_MSI, dtype: int64

#### TMB

In [170]:
# TMB
df_gene['TMB']=df_gene['TMB'].apply(lambda x: np.nan if pd.isnull(x) else
                                            np.nan if not re.match('.*?\d.*?',str(x)) else
                                            re.findall('\d\.\d',str(x))[0])

In [171]:
df_gene['TMB'].value_counts()

0.5    1
1.7    1
2.8    1
1.4    1
6.1    1
1.5    1
7.7    1
Name: TMB, dtype: int64

In [172]:
df_gene.to_excel(project_path+'/data/processed_data/df_1.8.5_基因指标.xlsx')

## 免疫组化

In [173]:
df_IHC=df_gene.copy()

In [174]:
IHC_col=['CDX-2','CEA','CK5/6','CK7','CK19','CK20','MUC-1','MOC-31','PD-1','PD-L1(DAKO,22C3)']
# 将可疑阳性0.5替换1
for i in IHC_col:
    df_IHC[i]=df_IHC[i].apply(lambda x: np.nan if pd.isnull(x) else 1 if str(x)=='0.5' else x)

In [175]:
# 重命名
df_IHC=df_IHC.rename(columns={'CDX-2':'IHC_cdx2',
                              'CEA':'IHC_cea',
                              'CK5/6':'IHC_ck5',
                              'CK7':'IHC_ck7',
                              'CK19':'IHC_ck19',
                              'CK20':'IHC_ck20',
                              'MUC-1':'IHC_muc1',
                              'MOC-31':'IHC_moc31',
                              'PD-1':'IHC_pd1',
                              'PD-L1(DAKO,22C3)':'IHC_pdl1'})

### PD-L1

In [176]:
'''
PD-L1
标签：阴性--0，表示pdl1<1%; 阳性--1，表示pdl1>=1%
'''
df_IHC['IHC_pdl1']=df_IHC['IHC_pdl1'].apply(lambda x: np.nan if pd.isnull(x) else
                                                      0 if re.match('.*<1%|阴性.*?',str(x)) else
                                                      1 if re.match('.*?阳性.*?',str(x)) else
                                                      0 if float(x)<0.01 else 
                                                      1 if float(x)>=0.01 else np.nan)

In [177]:
df_IHC['IHC_pdl1'].value_counts()

0.0    15
1.0     3
Name: IHC_pdl1, dtype: int64

In [178]:
df_IHC.to_excel(project_path+'/data/processed_data/df_1.9_免疫组化.xlsx')

# 目标变量

In [179]:
df_target=df_IHC.copy()

## 术后并发症

In [180]:
# 围手术期并发症【Clavien-Dindo分级系统】（I级、II级、IIIa级、IIIb级、IVa级、IVb级、V级）
df_target.rename(columns={'围手术期并发症【Clavien-Dindo分级系统】（I级、II级、IIIa级、IIIb级、IVa级、IVb级、V级）':'target_POC'},inplace=True)
df_target['target_POC']=df_target['target_POC'].apply(lambda x: np.nan if pd.isnull(x) else 
                                                        np.nan if str(x)=='未手术' else
                                                        0 if str(x)=='0' else
                                                        1 if str(x)=='1' else
                                                        5 if re.match('V|Ⅴ',str(x)) else
                                                        4 if re.match('.*?Ⅳ|IV.*?',str(x)) else
                                                        3 if re.match('.*?Ⅲ|III.*?',str(x)) else
                                                        2 if re.match('.*?Ⅱ|II.*?',str(x)) else
                                                        1 if re.match('.*?I|Ⅰ.*?',str(x)) else np.nan)

In [181]:
df_target['target_POC'].value_counts()

1.0    240
2.0     63
3.0     49
4.0     18
5.0     15
0.0     10
Name: target_POC, dtype: int64

## 围手术期死亡

In [182]:
# 围手术期死亡【术后90天内】（0=无；1=死亡）
df_target.rename(columns={'围手术期死亡【术后90天内】（0=无；1=死亡）':'target_POD'},inplace=True)
df_target['target_POD']=df_target['target_POD'].apply(lambda x: np.nan if pd.isnull(x) else 
                                                                0 if re.match('0',str(x)) else
                                                                1 if re.match('1',str(x)) else
                                                                1 if re.match('.*?死亡.*?',str(x)) else x)

In [183]:
df_target['target_POD'].value_counts()

0.0    391
1.0     21
Name: target_POD, dtype: int64

In [184]:
df_target=df_target.drop(['target_POD'],axis=1)

## 后续治疗及随访

In [185]:
# 后续治疗及随访
df_target.rename(columns={'后续治疗及随访':'target_following'},inplace=True)

In [186]:
df_target=df_target.drop(['target_following'],axis=1)

In [187]:
df_target.shape

(483, 74)

## 目前是否死亡

In [188]:
# 目前是否生存（1=存活；2=死亡）
df_target.rename(columns={'目前是否生存（1=存活；2=死亡）':'target_death'},inplace=True)
df_target['target_death']=df_target['target_death'].apply(lambda x: np.nan if pd.isnull(x) else
                                                                1 if re.match('生存\d+?.*?',str(x)) else
                                                                0 if re.match('1',str(x)) else
                                                                1 if re.match('2',str(x)) else np.nan)

In [189]:
df_target['target_death'].value_counts()

1.0    379
0.0    103
Name: target_death, dtype: int64

## 术后生存时间

In [190]:
df_target.rename(columns={'生存时间(天)':'target_survival_day'},inplace=True)
df_target.rename(columns={'生存时间(月)':'target_survival_month'},inplace=True)

In [191]:
df_target.shape

(483, 74)

In [192]:
df_target.to_excel(project_path+'/data/processed_data/df_2_目标变量处理.xlsx')

# 划分数据集

In [261]:
df_dataset=df_target.copy()

## 术后预后数据集

In [262]:
'''
删除并发症、存活时间(天)
删除target_death、sampling缺失
删除1个月内死亡
删除超过1个月手术相关的死亡(4,105,312,344)
cox回归标签：
    1年生存期
    3年生存期
model标签：
    1，一年以内死亡
    2，1-3年死亡
    3，3-8年死亡
    4，未死亡
'''

'\n删除并发症、存活时间(天)\n删除target_death、sampling缺失\n删除1个月内死亡\n删除超过1个月手术相关的死亡(4,105,312,344)\ncox回归标签：\n    1年生存期\n    3年生存期\nmodel标签：\n    1，一年以内死亡\n    2，1-3年死亡\n    3，3-8年死亡\n    4，未死亡\n'

In [263]:
# 去除并发症、target_survival_day
df_dataset_survival=df_dataset.drop(['target_POC','target_survival_day'],axis=1)
# 删除target_death、sampling缺失
df_dataset_survival=df_dataset_survival[df_dataset_survival.target_death.notnull() & df_dataset_survival.sampling.notnull()]
# 删除少于1个月的生存时间
df_dataset_survival=df_dataset_survival[~ (df_dataset_survival.target_survival_month<=1)]

In [264]:
df_dataset_survival.shape

(454, 72)

In [265]:
# 删除超过1个月的手术相关死亡。id：105, 312, 344
del_surg_id=[4,105,312,344]
df_dataset_survival=df_dataset_survival[~ df_dataset_survival['id'].isin(del_surg_id)]

In [266]:
df_dataset_survival.shape

(451, 72)

In [267]:
df_dataset_survival.target_death.isnull().any()

False

### 术后预后风险-cox数据集

In [268]:
# 术后数据
df_dataset_survival_cox=df_dataset_survival.copy()
df_dataset_survival_cox=df_dataset_survival_cox.reset_index(drop=True)

In [269]:
'''
cox生存分析模型：
计算1年、3年、8年生存期和标签
'''

'\ncox生存分析模型：\n计算1年、3年、8年生存期和标签\n'

In [270]:
# 计算cox-1年生存期的标签和生存期
for i in range(df_dataset_survival_cox.shape[0]):
    # 判断患者是否死亡
    if df_dataset_survival_cox.loc[i,'target_death']==0:
        df_dataset_survival_cox.loc[i,'target_1Y_death']=0
        df_dataset_survival_cox.loc[i,'target_1Y_survival']=12
    elif df_dataset_survival_cox.loc[i,'target_death']==1:
        # 判断患者是否在1年内死亡
        df_dataset_survival_cox.loc[i,'target_1Y_death']=1
        survival_month=df_dataset_survival_cox.loc[i,'target_survival_month']
        if  survival_month <=12:
            df_dataset_survival_cox.loc[i,'target_1Y_survival']=survival_month

In [271]:
# 计算cox-3年生存期的标签和生存期
for i in range(df_dataset_survival_cox.shape[0]):
    # 判断患者是否死亡
    if df_dataset_survival_cox.loc[i,'target_death']==0:
        df_dataset_survival_cox.loc[i,'target_3Y_death']=0
        df_dataset_survival_cox.loc[i,'target_3Y_survival']=36
    elif df_dataset_survival_cox.loc[i,'target_death']==1:
        # 判断患者是否在1年内死亡
        df_dataset_survival_cox.loc[i,'target_3Y_death']=1
        survival_month=df_dataset_survival_cox.loc[i,'target_survival_month']
        if  survival_month <=36:
            df_dataset_survival_cox.loc[i,'target_3Y_survival']=survival_month

In [272]:
# 计算cox-5年生存期的标签和生存期
for i in range(df_dataset_survival_cox.shape[0]):
    # 判断患者是否死亡
    if df_dataset_survival_cox.loc[i,'target_death']==0:
        df_dataset_survival_cox.loc[i,'target_5Y_death']=0
        df_dataset_survival_cox.loc[i,'target_5Y_survival']=60
    elif df_dataset_survival_cox.loc[i,'target_death']==1:
        # 判断患者是否在1年内死亡
        df_dataset_survival_cox.loc[i,'target_5Y_death']=1
        survival_month=df_dataset_survival_cox.loc[i,'target_survival_month']
        if  survival_month <=60:
            df_dataset_survival_cox.loc[i,'target_5Y_survival']=survival_month

In [273]:
# 计算cox-8年生存期的标签和生存期
for i in range(df_dataset_survival_cox.shape[0]):
    # 判断患者是否死亡
    if df_dataset_survival_cox.loc[i,'target_death']==0:
        df_dataset_survival_cox.loc[i,'target_8Y_death']=0
        df_dataset_survival_cox.loc[i,'target_8Y_survival']=96
    elif df_dataset_survival_cox.loc[i,'target_death']==1:
        # 判断患者是否在1年内死亡
        df_dataset_survival_cox.loc[i,'target_8Y_death']=1
        survival_month=df_dataset_survival_cox.loc[i,'target_survival_month']
        if  survival_month <=96:
            df_dataset_survival_cox.loc[i,'target_8Y_survival']=survival_month

In [274]:
df_dataset_survival_cox.shape

(451, 80)

In [275]:
df_dataset_survival_cox.to_excel(project_path+'/data/processed_data/df_3.1.1_术后预后cox数据集.xlsx')

### 术后预后生存-model数据集

In [276]:
# 术后数据
df_dataset_survival_model=df_dataset_survival.copy()
df_dataset_survival_model=df_dataset_survival_model.reset_index(drop=True)

In [277]:
'''
预后生存时长预测模型：打标签
死亡：1年以内--1，1-3年--2，3-8年--3，未死亡--0
'''
df_dataset_survival_model['label']=df_dataset_survival_model.target_survival_month.apply(lambda x: np.nan if pd.isnull(x) else
                                                                                       1 if int(x)<12 else
                                                                                       2 if 12<=int(x)<36 else
                                                                                       3 if 36<=int(x)<=96 else np.nan)
# 未死亡标签4
for i in range(df_dataset_survival_model.shape[0]):
    if df_dataset_survival_model.loc[i,'target_death']==0:
        df_dataset_survival_model.loc[i,'label']=0

In [278]:
df_dataset_survival_model.shape

(451, 73)

In [279]:
# 删除标签缺失
df_dataset_survival_model=df_dataset_survival_model[df_dataset_survival_model.label.notnull()]
# 建模的时候要剔除未手术部分！！
df_dataset_survival_model=df_dataset_survival_model[df_dataset_survival_model['sampling']==1]

In [280]:
df_dataset_survival_model.shape

(419, 73)

In [281]:
df_dataset_survival_model.to_excel(project_path+'/data/processed_data/df_3.1.2_术后预后model数据集.xlsx')

## 术前预后数据集

In [282]:
# 术后变量
post_surgery_col=['AJCC_8','surgery_bleeding','surgery_CRCS','surgery_plasm','surgery_CP','surgery_result','IHC_cdx2',
                  'IHC_cea','IHC_ck5','IHC_ck7','IHC_ck19','IHC_ck20','IHC_muc1','IHC_moc31','IHC_pd1','IHC_pdl1']

In [283]:
len(post_surgery_col)

16

### 术前预后风险-cox数据集

In [284]:
df_dataset_survival_cox_before = df_dataset_survival_cox.drop(post_surgery_col,axis=1)

In [285]:
df_dataset_survival_cox_before.shape

(451, 64)

In [286]:
df_dataset_survival_cox_before.to_excel(project_path+'/data/processed_data/df_3.2.1_术前预后cox数据集.xlsx')

### 术前预后生存-model数据集

In [287]:
df_dataset_survival_model_after=df_dataset_survival_model.drop(post_surgery_col,axis=1)

In [288]:
df_dataset_survival_model_after.shape

(419, 57)

In [289]:
df_dataset_survival_model_after.to_excel(project_path+'/data/processed_data/df_3.2.2_术前预后model数据集.xlsx')

## 术后并发症数据集

In [290]:
'''
删除存活时间
删除缺失
删除并发症--0、5
删除未手术记录
'''

'\n删除存活时间\n删除缺失\n删除并发症--0、5\n删除未手术记录\n'

In [291]:
# 去除target_death, target_survival_day, target_survival_month
df_dataset_POC=df_dataset.drop(['target_death','target_survival_day','target_survival_month'],axis=1)
# 删除缺失
df_dataset_POC=df_dataset_POC[df_dataset_POC.target_POC.notnull()]
# 删除并发症--0,5
df_dataset_POC=df_dataset_POC[~ df_dataset_POC.target_POC.isin([0,5])]
# 删除未手术记录
df_dataset_POC=df_dataset_POC[df_dataset_POC.sampling==1]

In [292]:
df_dataset_POC.shape

(369, 71)

In [293]:
print(df_dataset_POC.target_POC.value_counts())

1.0    239
2.0     63
3.0     49
4.0     18
Name: target_POC, dtype: int64


In [294]:
df_dataset_POC.to_excel(project_path+'/data/processed_data/df_3.3_术后并发症数据集.xlsx')